In [1]:
import pickle
import time
from typing import List
import datetime as dt

import numpy as np
from tqdm import tqdm
import plotly.graph_objects as go
import plotly.express as px

from quara.objects.composite_system import CompositeSystem
from quara.objects.elemental_system import ElementalSystem
from quara.objects.matrix_basis import get_normalized_pauli_basis, get_pauli_basis, get_comp_basis
from quara.objects.povm import (
    Povm,
    get_x_measurement,
    get_y_measurement,
    get_z_measurement,
)
from quara.objects.qoperation import QOperation
from quara.objects.state import State, get_x0_1q,  get_x1_1q, get_y0_1q, get_y1_1q, get_z0_1q, get_z1_1q
from quara.protocol.qtomography.standard.standard_qst import StandardQst
from quara.protocol.qtomography.standard.standard_povmt import StandardPovmt
from quara.protocol.qtomography.standard.linear_estimator import LinearEstimator
from quara.protocol.qtomography.standard.projected_linear_estimator import ProjectedLinearEstimator

from quara.protocol.qtomography.standard.standard_qtomography_estimator import (
    StandardQTomographyEstimator,
    StandardQTomographyEstimationResult,
)

In [2]:
from numpy.testing import assert_allclose

In [3]:
from quara.data_analysis import data_analysis, physicality_violation_check

In [4]:
%load_ext autoreload
%autoreload 2

# 1 ">=": True "=I": True 

In [55]:
# setup system
e_sys = ElementalSystem(0, get_normalized_pauli_basis())
c_sys = CompositeSystem([e_sys])

In [56]:
# basis
c_sys.basis()

MatrixBasis(basis=[array([[0.70710678+0.j, 0.        +0.j],
       [0.        +0.j, 0.70710678+0.j]]), array([[0.        +0.j, 0.70710678+0.j],
       [0.70710678+0.j, 0.        +0.j]]), array([[0.+0.j        , 0.-0.70710678j],
       [0.+0.70710678j, 0.+0.j        ]]), array([[ 0.70710678+0.j,  0.        +0.j],
       [ 0.        +0.j, -0.70710678+0.j]])])

In [57]:
m_1 = np.array([1/np.sqrt(2), 0, 0, 0])
m_2 = np.array([1/np.sqrt(2), 0, 0, 0])

vecs = [m_1, m_2]
povm = Povm(c_sys=c_sys, vecs=vecs, is_physicality_required=False)

In [58]:
povm.to_stacked_vectortacked_vector()

array([0.70710678, 0.        , 0.        , 0.        , 0.70710678,
       0.        , 0.        , 0.        ])

In [41]:
result = povm.calc_proj_physical()

In [43]:
result.is_positive_semidefinite()

True

In [7]:
assert povm.is_positive_semidefinite() is True
assert povm.is_identity_sum() is True

In [45]:
result.is_positive_semidefinite()
result.is_identity_sum() 

True

In [10]:
print("calc_proj_ineq_constraint():")
proj_ineq_povm = povm.calc_proj_ineq_constraint()
print(f">=0: {proj_ineq_povm.is_positive_semidefinite()}")  # expected: True

print(f"vecs:")
print(proj_ineq_povm.vecs)
print(f"matrices:")
print(proj_ineq_povm.matrices())

print("---------------")

print("calc_proj_eq_constraint():")
proj_eq_povm = povm.calc_proj_eq_constraint()
print(f"=I: {proj_eq_povm.is_identity_sum()}")  # expected: True 

print(f"vecs:")
print(proj_eq_povm.vecs)
print(f"matrices:")
print(proj_eq_povm.matrices())

calc_proj_ineq_constraint():
>=0: True
vecs:
(array([0.70710678, 0.        , 0.        , 0.        ]), array([0.70710678, 0.        , 0.        , 0.        ]))
matrices:
[array([[0.5+0.j, 0. +0.j],
       [0. +0.j, 0.5+0.j]]), array([[0.5+0.j, 0. +0.j],
       [0. +0.j, 0.5+0.j]])]
---------------
calc_proj_eq_constraint():
=I: True
vecs:
(array([0.70710678, 0.        , 0.        , 0.        ]), array([0.70710678, 0.        , 0.        , 0.        ]))
matrices:
[array([[0.5+0.j, 0. +0.j],
       [0. +0.j, 0.5+0.j]]), array([[0.5+0.j, 0. +0.j],
       [0. +0.j, 0.5+0.j]])]


In [11]:
assert proj_ineq_povm.is_positive_semidefinite() is True
assert_allclose(povm.vecs, proj_ineq_povm.vecs)

In [12]:
assert proj_eq_povm.is_identity_sum() is True
assert_allclose(povm.vecs, proj_eq_povm.vecs)

# 2 ">=": False "=I": True 

In [59]:
m_1 = np.array([np.sqrt(2), 0., 0., np.sqrt(2)])
m_2 = np.array([0, 0., 0., -np.sqrt(2)])

vecs = [m_1, m_2]
povm = Povm(c_sys=c_sys, vecs=vecs, is_physicality_required=False)

In [61]:
povm.to_stacked_vector()

array([ 1.41421356,  0.        ,  0.        ,  1.41421356,  0.        ,
        0.        ,  0.        , -1.41421356])

In [47]:
assert povm.is_positive_semidefinite() is False
assert povm.is_identity_sum() is True

In [48]:
result = povm.calc_proj_physical()
print(result.is_positive_semidefinite())
print(result.is_identity_sum())

False
True


In [15]:
print("calc_proj_ineq_constraint():")
proj_ineq_povm = povm.calc_proj_ineq_constraint()
print(f">=0: {proj_ineq_povm.is_positive_semidefinite()}")  # expected: True

print(f"vecs:")
print(proj_ineq_povm.vecs)
print(f"matrices:")
print(proj_ineq_povm.matrices())

print("---------------")

print("calc_proj_eq_constraint():")
proj_eq_povm = povm.calc_proj_eq_constraint()
print(f"=I: {proj_eq_povm.is_identity_sum()}")  # expected: True 

print(f"vecs:")
print(proj_eq_povm.vecs)
print(f"matrices:")
print(proj_eq_povm.matrices())

calc_proj_ineq_constraint():
>=0: True
vecs:
(array([1.41421356, 0.        , 0.        , 1.41421356]), array([ 0.70710678,  0.        ,  0.        , -0.70710678]))
matrices:
[array([[2.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j]]), array([[0.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j]])]
---------------
calc_proj_eq_constraint():
=I: True
vecs:
(array([1.41421356, 0.        , 0.        , 1.41421356]), array([ 0.        ,  0.        ,  0.        , -1.41421356]))
matrices:
[array([[2.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j]]), array([[-1.+0.j,  0.+0.j],
       [ 0.+0.j,  1.+0.j]])]


In [16]:
assert proj_ineq_povm.is_positive_semidefinite() is True
assert not np.allclose(povm.vecs, proj_ineq_povm.vecs)

In [17]:
assert proj_eq_povm.is_identity_sum() is True
assert_allclose(povm.vecs, proj_eq_povm.vecs)

# 3 ">=": True "=I": False

In [49]:
m_1 = np.array([np.sqrt(2), 0., 0., 1.])
m_2 = np.array([np.sqrt(2), 0., 0., 1.])

vecs = [m_1, m_2]
povm = Povm(c_sys=c_sys, vecs=vecs, is_physicality_required=False)

In [50]:
assert povm.is_positive_semidefinite() is True
assert povm.is_identity_sum() is False

In [51]:
result = povm.calc_proj_physical()
print(result.is_positive_semidefinite())
print(result.is_identity_sum() )

True
True


In [20]:
print("calc_proj_ineq_constraint():")
proj_ineq_povm = povm.calc_proj_ineq_constraint()
print(f">=0: {proj_ineq_povm.is_positive_semidefinite()}")  # expected: True

print(f"vecs:")
print(proj_ineq_povm.vecs)
print(f"matrices:")
print(proj_ineq_povm.matrices())

print("---------------")

print("calc_proj_eq_constraint():")
proj_eq_povm = povm.calc_proj_eq_constraint()
print(f"=I: {proj_eq_povm.is_identity_sum()}")  # expected: True 

print(f"vecs:")
print(proj_eq_povm.vecs)
print(f"matrices:")
print(proj_eq_povm.matrices())

calc_proj_ineq_constraint():
>=0: True
vecs:
(array([1.41421356, 0.        , 0.        , 1.        ]), array([1.41421356, 0.        , 0.        , 1.        ]))
matrices:
[array([[1.70710678+0.j, 0.        +0.j],
       [0.        +0.j, 0.29289322+0.j]]), array([[1.70710678+0.j, 0.        +0.j],
       [0.        +0.j, 0.29289322+0.j]])]
---------------
calc_proj_eq_constraint():
=I: True
vecs:
(array([0.70710678, 0.        , 0.        , 0.        ]), array([0.70710678, 0.        , 0.        , 0.        ]))
matrices:
[array([[0.5+0.j, 0. +0.j],
       [0. +0.j, 0.5+0.j]]), array([[0.5+0.j, 0. +0.j],
       [0. +0.j, 0.5+0.j]])]


In [21]:
assert proj_ineq_povm.is_positive_semidefinite() is True
assert_allclose(povm.vecs, proj_ineq_povm.vecs)

In [28]:
assert proj_eq_povm.is_identity_sum() is True
assert not np.allclose(povm.vecs, proj_eq_povm.vecs)

# 4 ">=": False "=I": False

In [52]:
m_1 = np.array([np.sqrt(2), 0., 0., 1.])
m_2 = np.array([-np.sqrt(2), 0., 0., 1.])

vecs = [m_1, m_2]
povm = Povm(c_sys=c_sys, vecs=vecs, is_physicality_required=False)

In [53]:
assert povm.is_positive_semidefinite() is False
assert povm.is_identity_sum() is False

In [54]:
result = povm.calc_proj_physical()
print(result.is_positive_semidefinite())
print(result.is_identity_sum() )

False
True


In [25]:
print("calc_proj_ineq_constraint():")
proj_ineq_povm = povm.calc_proj_ineq_constraint()
print(f">=0: {proj_ineq_povm.is_positive_semidefinite()}")  # expected: True

print(f"vecs:")
print(proj_ineq_povm.vecs)
print(f"matrices:")
print(proj_ineq_povm.matrices())

print("---------------")

print("calc_proj_eq_constraint():")
proj_eq_povm = povm.calc_proj_eq_constraint()
print(f"=I: {proj_eq_povm.is_identity_sum()}")  # expected: True 

print(f"vecs:")
print(proj_eq_povm.vecs)
print(f"matrices:")
print(proj_eq_povm.matrices())

calc_proj_ineq_constraint():
>=0: True
vecs:
(array([1.41421356, 0.        , 0.        , 1.        ]), array([0., 0., 0., 0.]))
matrices:
[array([[1.70710678+0.j, 0.        +0.j],
       [0.        +0.j, 0.29289322+0.j]]), array([[0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j]])]
---------------
calc_proj_eq_constraint():
=I: True
vecs:
(array([2.12132034, 0.        , 0.        , 0.        ]), array([-0.70710678,  0.        ,  0.        ,  0.        ]))
matrices:
[array([[1.5+0.j, 0. +0.j],
       [0. +0.j, 1.5+0.j]]), array([[-0.5+0.j,  0. +0.j],
       [ 0. +0.j, -0.5+0.j]])]


In [26]:
assert proj_ineq_povm.is_positive_semidefinite() is True
assert not np.allclose(povm.vecs, proj_ineq_povm.vecs)

In [27]:
assert proj_eq_povm.is_identity_sum() is True
assert not np.allclose(povm.vecs, proj_eq_povm.vecs)

In [62]:
# 

In [63]:
m_1 = np.array([1/np.sqrt(2), 0, 0, 0])
m_2 = np.array([1/np.sqrt(2), 0, 0, 0])

vecs = [m_1, m_2]
povm_1 = Povm(c_sys=c_sys, vecs=vecs, is_physicality_required=False)

m_1 = np.array([np.sqrt(2), 0., 0., np.sqrt(2)])
m_2 = np.array([0, 0., 0., -np.sqrt(2)])

vecs = [m_1, m_2]
povm_2 = Povm(c_sys=c_sys, vecs=vecs, is_physicality_required=False)

(povm_1 + povm_2).vecs

In [70]:
m_1 = np.array([1., 2, 3, 4])
m_2 = np.array([5., 6, 7, 8])

vecs = [m_1, m_2]
povm_1 = Povm(c_sys=c_sys, vecs=vecs, is_physicality_required=False)

m_1 = np.array([10., 20, 30, 40])
m_2 = np.array([50., 60, 70, 80])

vecs = [m_1, m_2]
povm_2 = Povm(c_sys=c_sys, vecs=vecs, is_physicality_required=False)

(povm_1 - povm_2).vecs

(array([ -9., -18., -27., -36.]), array([-45., -54., -63., -72.]))

In [75]:
m_1 = np.array([1., 2, 3, 4])
m_2 = np.array([5., 6, 7, 8])

vecs = [m_1, m_2]
povm_1 = Povm(c_sys=c_sys, vecs=vecs, is_physicality_required=False)


(povm_1 * 2).vecs

(array([2., 4., 6., 8.]), array([10., 12., 14., 16.]))